In [70]:
library(lmerTest)
library(MuMIn)
library(VIF)
library(sqldf)

filename <- 'generated/metrics_for_time_series.csv'

TimeSeriesData <- read.csv(file=filename, header=TRUE, sep=",")

summary(TimeSeriesData)

TimeSeriesData <- sqldf("select *
                      from 'TimeSeriesData' 
                      where Period != 13")


TimeSeriesData$Intervention = TimeSeriesData$Period > 12
TimeSeriesData$TimeAfterIntervention = ifelse(TimeSeriesData$Period>12, TimeSeriesData$Period-12, 0)

                         ProjectName         Language    ShareReviewComments
 activemerchant/active_merchant:  25   Ruby      :1200   Min.   :0.0000     
 adobe/brackets                :  25   Python    :1125   1st Qu.:0.0000     
 AFNetworking/AFNetworking     :  25   JavaScript: 925   Median :0.1186     
 airbnb/javascript             :  25   PHP       : 800   Mean   :0.1855     
 ajaxorg/ace                   :  25   C++       : 775   3rd Qu.:0.3090     
 alexreisner/geocoder          :  25   Java      : 700   Max.   :1.0000     
 (Other)                       :7450   (Other)   :2075                      
   Additions        Deletions       ChangedFiles      Assignees      
 Min.   : 0.000   Min.   :0.0000   Min.   :0.0000   Min.   :0.00000  
 1st Qu.: 1.586   1st Qu.:0.8815   1st Qu.:0.6931   1st Qu.:0.00000  
 Median : 2.382   Median :1.4915   Median :1.0104   Median :0.00000  
 Mean   : 2.423   Mean   :1.5884   Mean   :1.0583   Mean   :0.04135  
 3rd Qu.: 3.200   3rd Qu.:2.1859  

In [40]:
vif.mer <- function (fit) {
    ## adapted from rms::vif
    
    v <- vcov(fit)
    nam <- names(fixef(fit))

    ## exclude intercepts
    ns <- sum(1 * (nam == "Intercept" | nam == "(Intercept)"))
    if (ns > 0) {
        v <- v[-(1:ns), -(1:ns), drop = FALSE]
        nam <- nam[-(1:ns)]
    }
    
    d <- diag(v)^0.5
    v <- diag(solve(v/(d %o% d)))
    names(v) <- nam
    v
}

In [41]:
projects <- sqldf("select count(distinct(ProjectName)) from TimeSeriesData")

print(projects)

  count(distinct(ProjectName))
1                          304


In [76]:
modelNumberReviewComments = lmer(ReviewComments ~ 
            Additions+
            Deletions  +
            Commits  +
            #CommitsAfterCreate  +
            Assignees  + 
            ChangedFiles + 
            TotalPrs +
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1|Language) +
            (1+Intervention|ProjectName),
          data= TimeSeriesData, 
          REML=FALSE)
summary(modelNumberReviewComments)
r.squaredGLMM(modelNumberReviewComments)
vif.mer(modelNumberReviewComments)
anova(modelNumberReviewComments)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: ReviewComments ~ Additions + Deletions + Commits + Assignees +  
    ChangedFiles + TotalPrs + Intervention + Period + TimeAfterIntervention +  
    (1 | Language) + (1 + Intervention | ProjectName)
   Data: TimeSeriesData

     AIC      BIC   logLik deviance df.resid 
 -1428.9  -1325.4    729.4  -1458.9     7281 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.0023 -0.4528 -0.0819  0.3420 12.0942 

Random effects:
 Groups      Name             Variance Std.Dev. Corr 
 ProjectName (Intercept)      0.03424  0.18504       
             InterventionTRUE 0.01666  0.12907  -0.09
 Language    (Intercept)      0.00179  0.04231       
 Residual                     0.03997  0.19991       
Number of obs: 7296, groups:  ProjectName, 304; Language, 36

Fixed effects:
                        Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)  

R2m        R2c 
0.08694178 0.55547910

Additions             Deletions               Commits 
             3.852010              2.730739              1.310668 
            Assignees          ChangedFiles              TotalPrs 
             1.009215              4.394933              1.020091 
     InterventionTRUE                Period TimeAfterIntervention 
             2.039423              4.070799              3.357702

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
Additions,4.99089458,4.99089458,1,7145.084,124.8786956,0.000000e+00
Deletions,0.02682426,0.02682426,1,7085.995,0.6711779,4.126688e-01
Commits,8.94443838,8.94443838,1,7216.460,223.8015208,0.000000e+00
Assignees,0.01654348,0.01654348,1,6434.702,0.4139395,5.199997e-01
ChangedFiles,1.22701357,1.22701357,1,7185.965,30.7014809,3.115933e-08
TotalPrs,0.24375318,0.24375318,1,5678.215,6.0990227,1.355486e-02
Intervention,0.08423634,0.08423634,1,1202.994,2.1077032,1.468199e-01
Period,2.11071035,2.11071035,1,6705.739,52.8127274,4.085621e-13
TimeAfterIntervention,0.57356183,0.57356183,1,6687.504,14.3512654,1.530087e-04


In [75]:
modelShareReviewComments = lmer(GeneralComments  ~ 
            Additions  +
            Deletions + 
            Commits + 
            Assignees +
            ChangedFiles +
            TotalPrs +
            #CommitsAfterCreate +
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1|Language) +
            (1+Intervention|ProjectName),
          data= TimeSeriesData, 
          REML=FALSE)
summary(modelShareReviewComments)
r.squaredGLMM(modelShareReviewComments)
vif.mer(modelShareReviewComments)
anova(modelShareReviewComments)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: GeneralComments ~ Additions + Deletions + Commits + Assignees +  
    ChangedFiles + TotalPrs + Intervention + Period + TimeAfterIntervention +  
    (1 | Language) + (1 + Intervention | ProjectName)
   Data: TimeSeriesData

     AIC      BIC   logLik deviance df.resid 
  4337.3   4440.7  -2153.6   4307.3     7281 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.3346 -0.5442 -0.0226  0.5107  6.4213 

Random effects:
 Groups      Name             Variance Std.Dev. Corr 
 ProjectName (Intercept)      0.11103  0.3332        
             InterventionTRUE 0.05460  0.2337   -0.26
 Language    (Intercept)      0.00000  0.0000        
 Residual                     0.08589  0.2931        
Number of obs: 7296, groups:  ProjectName, 304; Language, 36

Fixed effects:
                        Estimate Std. Error         df t value Pr(>|t|)    
(Intercept) 

R2m        R2c 
0.05966071 0.60421846

Additions             Deletions               Commits 
             3.829179              2.706364              1.305450 
            Assignees          ChangedFiles              TotalPrs 
             1.008931              4.365646              1.017909 
     InterventionTRUE                Period TimeAfterIntervention 
             1.755925              3.513655              2.989168

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
Additions,5.1859210,5.1859210,1,7059.5324,60.38043,8.881784e-15
Deletions,1.2085223,1.2085223,1,7011.7991,14.07100,1.774564e-04
Commits,14.1554108,14.1554108,1,7150.2411,164.81350,0.000000e+00
Assignees,4.1669915,4.1669915,1,6886.0245,48.51689,3.577583e-12
ChangedFiles,1.5311067,1.5311067,1,7106.6024,17.82690,2.449551e-05
TotalPrs,1.2847543,1.2847543,1,6345.4737,14.95858,1.109965e-04
Intervention,0.9192769,0.9192769,1,906.5334,10.70327,1.109841e-03
Period,3.8164149,3.8164149,1,6705.4045,44.43507,2.838574e-11
TimeAfterIntervention,1.5782344,1.5782344,1,6690.4157,18.37561,1.839104e-05


In [77]:
modelShareReviewComments = lmer(EffectiveComments ~ 
            Additions + 
            Deletions + 
            Commits + 
            Assignees + 
            ChangedFiles + 
            #GeneralComments + 
            TotalPrs +
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1|Language) +
            (1+Intervention|ProjectName),
          data= TimeSeriesData, 
          REML=FALSE)
summary(modelShareReviewComments)
r.squaredGLMM(modelShareReviewComments)
vif.mer(modelShareReviewComments)
anova(modelShareReviewComments)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: EffectiveComments ~ Additions + Deletions + Commits + Assignees +  
    ChangedFiles + TotalPrs + Intervention + Period + TimeAfterIntervention +  
    (1 | Language) + (1 + Intervention | ProjectName)
   Data: TimeSeriesData

     AIC      BIC   logLik deviance df.resid 
-10200.7 -10097.3   5115.4 -10230.7     7281 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.9855 -0.3924 -0.0927  0.2136 21.5281 

Random effects:
 Groups      Name             Variance  Std.Dev. Corr 
 ProjectName (Intercept)      0.0053491 0.07314       
             InterventionTRUE 0.0032844 0.05731  -0.29
 Language    (Intercept)      0.0002394 0.01547       
 Residual                     0.0125468 0.11201       
Number of obs: 7296, groups:  ProjectName, 304; Language, 36

Fixed effects:
                        Estimate Std. Error         df t value Pr(>|t|)    
(Inte

R2m       R2c 
0.1049375 0.3948804

Additions             Deletions               Commits 
             3.947200              2.814461              1.319891 
            Assignees          ChangedFiles              TotalPrs 
             1.008965              4.490576              1.021025 
     InterventionTRUE                Period TimeAfterIntervention 
             2.415153              4.806177              3.845778

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
Additions,1.296957960,1.296957960,1,7275.612,103.3698477,0.000000e+00
Deletions,0.363749784,0.363749784,1,7223.170,28.9915024,7.498454e-08
Commits,5.310722393,5.310722393,1,7250.522,423.2739855,0.000000e+00
Assignees,0.010527377,0.010527377,1,4204.098,0.8390506,3.597210e-01
ChangedFiles,0.239565637,0.239565637,1,7269.723,19.0938058,1.261778e-05
TotalPrs,0.005036171,0.005036171,1,3616.386,0.4013917,5.264114e-01
Intervention,0.006585597,0.006585597,1,1622.521,0.5248837,4.688703e-01
Period,0.391250747,0.391250747,1,6711.094,31.1833778,2.439582e-08
TimeAfterIntervention,0.267699412,0.267699412,1,6686.646,21.3361175,3.926174e-06


In [79]:
modelShareReviewComments = lmer(CommitsAfterCreate ~ 
            Additions + 
            Deletions + 
            Commits + 
            Assignees + 
            ChangedFiles + 
            TotalPrs +
            GeneralComments + 
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1|Language) +
            (1+Intervention|ProjectName),
          data= TimeSeriesData, 
          REML=FALSE)
summary(modelShareReviewComments)
r.squaredGLMM(modelShareReviewComments)
vif.mer(modelShareReviewComments)
anova(modelShareReviewComments)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: CommitsAfterCreate ~ Additions + Deletions + Commits + Assignees +  
    ChangedFiles + TotalPrs + GeneralComments + Intervention +  
    Period + TimeAfterIntervention + (1 | Language) + (1 + Intervention |  
    ProjectName)
   Data: TimeSeriesData

     AIC      BIC   logLik deviance df.resid 
 -4958.7  -4848.3   2495.3  -4990.7     7280 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-10.8614  -0.4345  -0.0085   0.4173  11.3471 

Random effects:
 Groups      Name             Variance Std.Dev. Corr 
 ProjectName (Intercept)      0.009398 0.09694       
             InterventionTRUE 0.006582 0.08113  -0.62
 Language    (Intercept)      0.000640 0.02530       
 Residual                     0.026253 0.16203       
Number of obs: 7296, groups:  ProjectName, 304; Language, 36

Fixed effects:
                        Estimate Std. Error        

R2m       R2c 
0.5475904 0.6580968

Additions             Deletions               Commits 
             4.041058              2.871336              1.348572 
            Assignees          ChangedFiles              TotalPrs 
             1.013819              4.547708              1.020562 
      GeneralComments      InterventionTRUE                Period 
             1.063692              2.456434              4.900280 
TimeAfterIntervention 
             3.901847

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
Additions,1.166546e+00,1.166546e+00,1,7230.328,4.443454e+01,2.823652e-11
Deletions,4.543434e-02,4.543434e-02,1,7249.877,1.730626e+00,1.883726e-01
Commits,1.054670e+02,1.054670e+02,1,6999.949,4.017311e+03,0.000000e+00
Assignees,2.730551e-01,2.730551e-01,1,2443.285,1.040086e+01,1.276184e-03
ChangedFiles,1.348665e+00,1.348665e+00,1,7130.359,5.137158e+01,8.411050e-13
TotalPrs,3.365115e-02,3.365115e-02,1,2125.484,1.281796e+00,2.576931e-01
GeneralComments,1.690303e+01,1.690303e+01,1,5072.985,6.438482e+02,0.000000e+00
Intervention,2.374365e-03,2.374365e-03,1,1645.993,9.044123e-02,7.636550e-01
Period,1.556662e-01,1.556662e-01,1,6735.611,5.929434e+00,1.491581e-02
TimeAfterIntervention,6.721950e-02,6.721950e-02,1,6699.351,2.560438e+00,1.096154e-01


# how about if we don't apply any aggregation

In [29]:
filename_no_aggr <- 'generated/metrics_for_time_series_no_aggr.csv'

TimeSeriesDataNoAggr <- read.csv(file=filename_no_aggr, header=TRUE, sep=",")

summary(TimeSeriesDataNoAggr)

TimeSeriesDataNoAggr <- sqldf("select *
                      from 'TimeSeriesDataNoAggr' 
                      where Period != 13")


TimeSeriesDataNoAggr$Intervention = TimeSeriesDataNoAggr$Period > 12
TimeSeriesDataNoAggr$TimeAfterIntervention = ifelse(TimeSeriesDataNoAggr$Period>12, TimeSeriesDataNoAggr$Period-12, 0)

                       ProjectName       Period      EffectiveComments
 adobe/brackets              : 638   Min.   : 1.00   Min.   :  1.000  
 caskdata/cdap               : 633   1st Qu.: 7.00   1st Qu.:  1.000  
 DynamoDS/Dynamo             : 420   Median :13.00   Median :  2.000  
 symfony/symfony-docs        : 370   Mean   :12.93   Mean   :  6.055  
 AnalyticalGraphicsInc/cesium: 261   3rd Qu.:19.00   3rd Qu.:  5.000  
 cakephp/docs                : 253   Max.   :25.00   Max.   :190.000  
 (Other)                     :1966                                    
 ReviewThreads    GeneralComments     Additions          Deletions       
 Min.   :  1.00   Min.   :  0.000   Min.   :     0.0   Min.   :     0.0  
 1st Qu.:  2.00   1st Qu.:  2.000   1st Qu.:    13.0   1st Qu.:     1.0  
 Median :  5.00   Median :  4.000   Median :    87.0   Median :    12.0  
 Mean   : 12.55   Mean   :  8.498   Mean   :   819.5   Mean   :   322.8  
 3rd Qu.: 13.00   3rd Qu.:  9.000   3rd Qu.:   347.0   3rd Qu.

In [30]:
projects <- sqldf("select count(distinct(ProjectName)) from TimeSeriesDataNoAggr")

print(projects)

  count(distinct(ProjectName))
1                           18


In [31]:
modelEffectiveCommentsNoAggr = lmer(log(EffectiveComments + 1) ~ 
            log(Additions + 1) +
            log(Deletions + 1) +
            log(Commits + 1) +
            log(Assignees + 1) + 
            log(ChangedFiles + 1) + 
            IsMerged + 
            Intervention +
            Period + 
            TimeAfterIntervention +
            (1+Intervention|ProjectName),
          data= TimeSeriesDataNoAggr, 
          REML=FALSE)
summary(modelEffectiveCommentsNoAggr)
r.squaredGLMM(modelEffectiveCommentsNoAggr)
vif.mer(modelEffectiveCommentsNoAggr)
anova(modelEffectiveCommentsNoAggr)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: log(EffectiveComments + 1) ~ log(Additions + 1) + log(Deletions +  
    1) + log(Commits + 1) + log(Assignees + 1) + log(ChangedFiles +  
    1) + IsMerged + Intervention + Period + TimeAfterIntervention +  
    (1 + Intervention | ProjectName)
   Data: TimeSeriesDataNoAggr

     AIC      BIC   logLik deviance df.resid 
  9171.4   9260.6  -4571.7   9143.4     4317 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5157 -0.6843 -0.1494  0.5464  4.4667 

Random effects:
 Groups      Name             Variance Std.Dev. Corr
 ProjectName (Intercept)      0.02523  0.1588       
             InterventionTRUE 0.01784  0.1336   0.00
 Residual                     0.47627  0.6901       
Number of obs: 4331, groups:  ProjectName, 18

Fixed effects:
                        Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)            3.615e-01  

R2m       R2c 
0.2533677 0.3031359

log(Additions + 1)    log(Deletions + 1)      log(Commits + 1) 
             3.185133              2.281099              1.398733 
   log(Assignees + 1) log(ChangedFiles + 1)          IsMergedTrue 
             1.009230              3.720643              1.036548 
     InterventionTRUE                Period TimeAfterIntervention 
             2.047772              3.804132              3.322282

,Sum Sq,Mean Sq,NumDF,DenDF,F.value,Pr(>F)
log(Additions + 1),75.04443963,75.04443963,1,4321.977,157.56617240,0.00000000
log(Deletions + 1),57.90042174,57.90042174,1,4317.630,121.56993748,0.00000000
log(Commits + 1),349.96244096,349.96244096,1,4319.894,734.79451083,0.00000000
log(Assignees + 1),4.23595771,4.23595771,1,3574.080,8.89397863,0.00288045
log(ChangedFiles + 1),2.82422692,2.82422692,1,4328.159,5.92985474,0.01492659
IsMerged,0.15024479,0.15024479,1,4303.772,0.31545971,0.57437830
Intervention,0.02744097,0.02744097,1,60.608,0.05761611,0.81111424
Period,2.46823576,2.46823576,1,4261.915,5.18240209,0.02286613
TimeAfterIntervention,2.09320811,2.09320811,1,4274.361,4.39497972,0.03610353
